https://www.kaggle.com/durbin164/lux-ai-rules-base-agent

In [3]:
!pip install kaggle-environments -U

In [4]:
!git clone https://github.com/Lux-AI-Challenge/Lux-Design-2021.git

Cloning into 'Lux-Design-2021'...
remote: Enumerating objects: 4025, done.
remote: Counting objects: 100% (1937/1937), done.
remote: Compressing objects: 100% (913/913), done.
remote: Total 4025 (delta 1162), reused 1691 (delta 993), pack-reused 2088
Receiving objects: 100% (4025/4025), 16.25 MiB | 16.37 MiB/s, done.
Resolving deltas: 100% (2404/2404), done.


In [5]:
!cp -r ./Lux-Design-2021/kits/python/simple/lux .

In [37]:
from lux.game import Game
from lux.game_map import Cell, RESOURCE_TYPES, Position
from lux.constants import Constants
from lux.game_constants import GAME_CONSTANTS
from lux import annotate
import math
import sys
import collections

import numpy as np

In [38]:
# game_state를 받아와서, resource가 있는 cell들을 list에 추가
def find_resources(game_state):
    resource_tiles: list[Cell] = []
    width, height = game_state.map_width, game_state.map_height
    for y in range(height):
        for x in range(width):
            cell = game_state.map.get_cell(x, y)
            if cell.has_resource():
                resource_tiles.append(cell)
    return resource_tiles

In [39]:
# 현재 포지션에서 해당 player가 닿을 수 있는 가장 가까운 resource tile
def find_closest_resources(pos, player, resource_tiles):
    closest_dist = main.inf
    closest_resource_tile = None
    for resource_tile in resource_tiles:
        if resource_tile.resource.type == Constants.RESOURCE_TYPES.COAL and not player.researched_coal():
            continue
        if resource_tile.resource.type == Constants.RESOURCE_TYPES.URANIUM and not player.researched_uranium():
            continue

        dist = resource_tile.pos.distance_to(pos)
        if dist < closest_dist:
            closest_dist = dist
            closest_resource_tile = resource_tile

    return closest_resource_tile

In [40]:
# 현재 위치에서 player에게 가장 가까운 city
def find_closest_city_tile(pos, player):
    closest_city_tile = None
    if len(player.cities) > 0:
        closest_dist = math.inf

        for k, city in player.cities.items():
            for city_tile in city.citytiles:
                dist = city_tile.pos.distance_to(pos)
                if dist < closest_dist:
                    closest_dist = dist
                    closest_city_tile = closest_city_tile

    return closest_city_tile

In [41]:
# resource tile 중에 player와 가까운 거리 순으로 정렬
def find_resources_distance(pos, player, resource_tiles):
    resources_distance = {}
    for resource_tile in resource_tiles:
        if resource_tile.resource.type == Constants.RESOURCE_TYPES.COAL and not player.researched_coal():
            continue
        if resource_tile.resource.type == Constants.RESOURCE_TYPES.URANIUM and not player.researched_uranium():
            continue

        dist = resource_tile.pos.distance_to(pos)
        resources_distance[resource_tile] = dist

    resource_distance = collections.OrderedDict(sorted(resources_distance.items(), key = lambda x:x[1]))
    return resource_distance

In [42]:
# city tile 중에 player와 가까운 거리 순으로 정렬
def find_city_tile_distance(pos, player):
    city_tiles_distance = {}
    if len(player.cities) > 0:
        for k, city in player.cities.items():
            for city_tile in city.citytiles:
                dist = city_tile.pos.distance_to(pos)
                city_tiles_distance[city_tile] = dist
    city_tiles_distance = collections.OrderedDict(sorted(city_tiles_distance.items(), key = lambda x: x[1]))
    return city_tiles_distance

In [43]:
def get_random_step():
    return np.random.choice(['s', 'n', 'w', 'e'])

In [64]:
game_state = None

def agent(observation, configuration):
    global game_state

    ### Do not edit ###
    if observation["step"] == 0:
        game_state = Game()
        game_state._initialize(observation["updates"])
        game_state._update(observation["updates"][2:])
        game_state.id = observation.player
    else:
        game_state._update(observation["updates"])

    actions = []

    player = game_state.players[observation.player]
    opponent = game_state.players[(observation.player + 1) % 2]
    width, height = game_state.map.width, game_state.map.height

    if game_state.turn == 0:
        print("Agent is running!", file=sys.stderr)

    # 현재 game 판에 있는 모든 resource tile 가져옴
    resource_tiles = find_resources(game_state)

    # unit의 다음 움직임을 저장해두는 dict
    move_mapper = {}

    # 현재 모든 city tile 수
    total_city_tiles = sum([len(city.citytiles) for city in player.cities.values()])

    # 만약 city tile이 현재 unit 보다 많으면 unit을 만든다.
    if total_city_tiles > len(player.units):
        for city in player.cities.values():
            city_tiles = city.citytiles[0]
            if city_tiles.cooldown <= 0:
                action = city_tiles.build_worker()
                actions.append(action)
                break

    # 남은 밤 수 
    night_step_left = 40 - max((observation["step"] % 40), 30)

    # 도시를 지을 수 있을지 없을지 판단
    can_build = False

    # 만약에 city가 하나라도 있고
    if len(player.cities) > 0:
        # 도시들을 loop 돌면서
        for k, city in player.cities.items():
            # 각 도시의 city tile의 수
            total_city_tiles = len(city.citytiles)

            # 각 도시를 유지하기 위해 필요한 fuel (이해가 잘 안감)
            total_need_fuel = (23 * total_city_tiles * night_step_left) * 3.5

            # 만약 도시의 연료가 필요 연료보다 많으면
            if city.fuel - total_need_fuel > 20:
                # 하나 더 지을 수 있다
                can_build = True
                break

    # 유닛의 현재 위치를 move_mapper에 삽입
    for unit in player.units:
        move_mapper[(unit.pos.x, unit.pos.y)] = unit

    # 각 유닛별로 loop
    for unit in player.units:
        # 유닛이 현재 움직일 수 있다면 (쿨타임이 끝났으면)
        if unit.is_worker() and unit.can_act():
            # 만약 도시를 지을 연료가 있고, 유닛의 현재 위치에다가 지을 수 있다면
            if can_build and unit.can_build(game_state.map):
                is_build = False
                
                # 유닛의 현재 위치 주변에 citytile이 있으면 지을 수 있다.
                for city in player.cities.values():
                    for citytiles in city.citytiles:
                        if citytiles.pos.is_adjacent(unit.pos):
                            is_build = True
                            break

                    if is_build:
                        break

                if is_build:
                    action = unit.build_city()
                    actions.append(action)

                    # 해당 유닛이 지을 거니까 can_build는 False 처리
                    can_build = False
                    continue

            # 만약 도시를 지을 수 없다면

            # 현재 유닛의 위치에서 resource tile들의 위치를 정렬해서 받아옴
            resources_distance = find_resources_distance(unit.pos, player, resource_tiles)
            # 현재 유닛의 위치에서 city tile들의 위치를 정렬해서 받아옴
            city_tile_distance = find_city_tile_distance(unit.pos, player)

            flag = True

            # 만약 유닛의 현재 위치가 resource tile 이면 flag 는 False
            for r in resource_tiles:
                if r.pos.equals(unit.pos):
                    flag = False
                    break

            # 유닛이 담을 수 있는 cago가 남아 있고, 현재 위치가 resource tile이 아니면
            if unit.get_cargo_space_left() > 0 and flag:
                # resource tile 이 있다면
                if resources_distance is not None and len(resources_distance) > 0:
                    closest_resource_tile, c_dist = None, None
                    can_move = False

                    # 각 resource tile을 loop
                    for resource, dist in resources_distance.items():
                        # 만약 해당 resource tile로 가고자 하는 유닛이 하나도 없으면
                        if move_mapper.get((resource.pos.x, resource.pos.y)) is None:
                            # 지금 resource가 가장 가까운 resource tile
                            closest_resource_tile = resource
                            c_dist = dist

                            # 만약 resource tile로 갈 수 있다면
                            if closest_resource_tile is not None and not closest_resource_tile.pos.equals(unit.pos):
                                # annotation line을 그리고
                                actions.append(annotate.line(unit.pos.x, unit.pos.y, closest_resource_tile.pos.x, closest_resource_tile.pos.y))
                                direction = unit.pos.direction_to(closest_resource_tile.pos)
                                
                                # 해당 resource tile로 갈 수 있는 다음 칸을 구함
                                next_pos = unit.pos.translate(direction, 1)

                                # 만약 갈 칸에 unit이 있으면 다음 resource tile을 찾아봄
                                if move_mapper.get((next_pos.x, next_pos.y)):
                                    continue

                                # 다음 칸으로 갈 수 있으면
                                can_move = True
                                next_state_pos = unit.pos.translate(direction, 1)

                                # 다음 칸으로 움직이고 mapper에 저장
                                action = unit.move(direction)
                                actions.append(action)
                                move_mapper[(next_state_pos.x, next_state_pos.y)] = unit
                                break

                    # 만약 갈 수 있는 칸이 없으면 아무렇게나 움직임
                    if not can_move:
                        direction = get_random_step()
                        next_state_pos = unit.pos.translate(direction, 1)

                        action = unit.move(direction)
                        actions.append(action)
                        move_mapper[(next_state_pos.x, next_state_pos.y)] = unit
            # 유닛이 담을 수 있는 연료 칸이 없으면
            else:
                # 만약 갈 수 있는 citytile이 있으면
                if city_tile_distance is not None and len(city_tile_distance) > 0:
                    closest_city_tile = None
                    can_move = False

                    # city tile 중에 가장 가까운 tile 선택
                    for city_tile, dist in city_tile_distance.items():
                        if move_mapper.get((city_tile.pos.x, city_tile.pos.y)) is None:
                            closest_city_tile = city_tile

                            if closest_city_tile is not None:
                                actions.append(annotate.line(unit.pos.x, unit.pos.y, closest_city_tile.pos.x, closest_city_tile.pos.y))
                                direction = unit.pos.direction_to(closest_city_tile.pos)
                                next_pos = unit.pos.translate(direction,1)

                                if move_mapper.get((next_pos.x, next_pos.y)):
                                    continue
                                
                                can_move = True
                                next_state_pos = unit.pos.translate(direction, 1)

                                action = unit.move(direction)
                                actions.append(action)
                                move_mapper[(next_state_pos.x, next_state_pos.y)] = unit

                                break
                    # 만약 갈 수 있는 city tile이 없으면 random
                    if not can_move:
                        direction = get_random_step()
                        next_state_pos = unit.pos.translate(direction, 1)

                        action = unit.move(direction)
                        actions.append(action)
                        move_mapper[(next_state_pos.x, next_state_pos.y)] = unit
    return actions

In [ ]:
from kaggle_environments import make

env = make("lux_ai_2021", configuration={"seed": 52, "loglevel": 2, "annotations": True}, debug=True)
steps = env.run([agent, "simple_agent"])
env.render(mode="ipython", width=1000, height=800)